# 2018 Col150 Data Set
## Data and Graphical Analysis

In [ ]:
import os, sys, shutil, subprocess
import scipy
from scipy import ndimage
from scipy.spatial import distance

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
matplotlib.rcParams.update({'figure.max_open_warning': 0})
matplotlib.rcParams.update({'font.size': 12})


%matplotlib inline
path = os.getcwd()

In [ ]:
def pca_df(filename):
    df   = pd.DataFrame(columns=['x','y','z'])
    infile = open(filename, 'r')
    rfdata = infile.readlines()
    infile.close()
    rfdata = [i.rstrip('\n').split() for i in rfdata]
    N = int(rfdata[0][0])
    for j in range(0, N):
        df.at[j, ['x','y','z']]  = rfdata[ 5*j + 2]

    df = df.astype('float')
    df['vx'] = df.x - df.x.mean()
    df['vy'] = df.y - df.y.mean()
    df['vz'] = df.z - df.z.mean()

    cxx = ( (df.vx*df.vx).sum() - (df.vx.sum() * df.vx.sum()) ) / (N-1)
    cxy = ( (df.vx*df.vy).sum() - (df.vx.sum() * df.vy.sum()) ) / (N-1)
    cxz = ( (df.vx*df.vz).sum() - (df.vx.sum() * df.vz.sum()) ) / (N-1)
    cyx = ( (df.vy*df.vx).sum() - (df.vy.sum() * df.vx.sum()) ) / (N-1)
    cyy = ( (df.vy*df.vy).sum() - (df.vy.sum() * df.vy.sum()) ) / (N-1)
    cyz = ( (df.vy*df.vz).sum() - (df.vy.sum() * df.vz.sum()) ) / (N-1)
    czx = ( (df.vz*df.vx).sum() - (df.vz.sum() * df.vx.sum()) ) / (N-1)
    czy = ( (df.vz*df.vy).sum() - (df.vz.sum() * df.vy.sum()) ) / (N-1)
    czz = ( (df.vz*df.vz).sum() - (df.vz.sum() * df.vz.sum()) ) / (N-1)

    covar = np.matrix([ [cxx, cxy, cxz], [cyx, cyy, cyz], [czx, czy, czz] ])
    evals, evecs = np.linalg.eig( covar )
    idx = np.argsort(evals)[::-1][: len(evals)]
    evals = evals[idx]
    evecs = evecs[:, idx]

    adj = pd.DataFrame(np.dot( df[['vx', 'vy', 'vz']], evecs), columns=['e1','e2','e3'])
    del rfdata, df
    return adj

def fig_move(figdirectory):
    for filename in os.listdir(figdirectory):
        if not os.path.exists("figures/data"):
            os.mkdir("figures/data")
    for filename in os.listdir(figdirectory):
        if filename.endswith(".png"):
            shutil.move(figdirectory+'/'+filename, figdirectory+"/figures/data/"+filename)
    return


In [ ]:
Ncirc = 150
ematrix_dict = {0:'tilt', 1:'roll', 2:'twist', 3:'shift', 4:'slide', 5:'rise'}

incons   = ['oring','pcirc']

inseqs  = ['col'+str(i).zfill(2) for i in range(1, 10, 1)]
inseqs1 = ['$A_{150}$','$G_{150}$','$A_{75}G_{75}$','$A_{15}G_{15}$','$A_1G_1$','$A_2G_2$','$A_3G_3$','$A_4G_4$','$A_5G_5$']
inseqs2 = ['A150','G150','A75G75','A15G15','A1G1','A2G2','A3G3','A4G4','A5G5']

#ff_ind         = ['ideal','kabsch','olson']
#ff_ind_colors  = {'ideal':'red','kabsch':'dodgerblue','olson':'goldenrod'}
#ff_ind_marks   = {'ideal':'.','kabsch':'^','olson':'d'}

ff_st         = [str(i)+'-state' for i in range(1, 5, 1)]
ff_dict       = {'idt':'1-state','org':'2-state','vic':'3-state','vor':'4-state'}
ff_st_dict    = {'1-state':'Ideal B-DNA Model','2-state':'O-Ring Model','3-state':'Zhurkin/Trifonov Model','4-state':'Zhurkin+O-Ring/Trifonov Model'}
ff_st_colors  = {'1-state':'goldenrod','2-state':'dodgerblue','3-state':'limegreen','4-state':'tomato'}
ff_st_marks   = {'1-state':'.','2-state':'8','3-state':'d','4-state':'s'}
ff_st_lines   = {'1-state':':','2-state':'--','3-state':':','4-state':'--'}

rs_bpt    = ['090','091','092','093','094','095','096','097','098','099','100','101','012','103','104','105','106','107','108','109','110']
rs_bpturn = [9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0,10.1,10.2,10.3,10.4,10.5,10.6,10.7,10.8,10.9,11.0]
rs_tw     = [round(float(360/i), 2) for i in rs_bpturn]

#hatchs = {'oring':"x", "pcirc":"o"}
plots  = ['eopt','Lk','Wr','Rg']
ylab   = {'eopt':'Optimized Energy ($k_B$T)',
          'Lk':'Linking Number',
          'Wr':'Geometric Writhe',
          'Rg':'Radius of Gyration ($\AA$)'}
ylimits = {'eopt':[0, 75],
          'Lk':[10, 20],
          'Wr':[-0.5, 0.5],
          'Rg':[45, 85]}
yax_minors = {'eopt':5, 'Lk':1, 'Wr':0.025, 'Rg':1}

horline ={'oring':{'eopt':31.585427,'Lk':15.0,'Wr':1.078891e-10,'Rg':81.17491},
          'pcirc':{'eopt':20.548522,'Lk':14.0,'Wr':-6.517982e-11,'Rg':81.174948}
         }

inseq_colorlist = ['red','green','darkorchid','darkgoldenrod', 'darkturquoise','violet','slategrey','darkorange','limegreen']
inseq_markers   = ['+','x','*','s','^','<','v','>','d']
marks           = {inseqs[i]:inseq_markers[i] for i in range(0, len(inseqs))}
colors          = {inseqs[i]:inseq_colorlist[i] for i in range(0, len(inseqs))}
labels          = {inseqs[i]:inseqs1[i] for i in range(0, len(inseqs))}

## Load Global Data .csv file and analyze properties

In [ ]:
# Plot eopt, lk, wr, and rg for the 4 deltaTw Models

col150_df = pd.read_csv("data_col150_seq-primary_standardset", index_col=0)

df0o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[0])]
df1o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[1])]
df2o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[2])]
df3o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[3])]

df0p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[0])]
df1p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[1])]
df2p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[2])]
df3p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[0])&(col150_df.forcefield==ff_st[3])]

for j in range(0, len(plots)):
    fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14,5), sharey=True)
    df0o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]], style=ff_st_lines[ff_st[0]],
              label=ff_st[0], legend=False, ax=ax0)
    df1o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]], style=ff_st_lines[ff_st[1]],
              label=ff_st[1], legend=False, ax=ax0)
    df2o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]], style=ff_st_lines[ff_st[2]],
              label=ff_st[2], legend=False, ax=ax0)
    df3o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]], style=ff_st_lines[ff_st[3]],
              label=ff_st[3], legend=False, ax=ax0)
    ax0.set_xticks(df0o.tw)
    
    df0p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]],  style=ff_st_lines[ff_st[0]],
              label=ff_st[0], legend=False, ax=ax1)
    df1p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]],  style=ff_st_lines[ff_st[1]],
              label=ff_st[1], legend=False, ax=ax1)
    df2p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]],  style=ff_st_lines[ff_st[2]],
              label=ff_st[2], legend=False, ax=ax1)
    df3p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]],  style=ff_st_lines[ff_st[3]],
              label=ff_st[3], legend=False, ax=ax1)
    ax1.set_xticks(df0p.tw)
    
    for ax in [ax0, ax1]:
        ax.yaxis.set_minor_locator(plt.MultipleLocator(yax_minors[plots[j]]))
        ax.set_xticklabels([s for s in rs_bpturn], rotation=60, fontsize=12)
        ax.set_ylabel(ylab[plots[j]], fontsize=12)
        ax.set_xlabel('Base-Pairs / Helical Turn')
        ax.grid(axis='y',which='both')
    
    plt.legend(labels=[ff_st_dict[x] for x in ff_st], loc='center left', bbox_to_anchor=(1.05, 0.50))
    plt.tight_layout()
    plt.show()
    plt.clf()

for j in range(0, len(plots)):
    fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14,5), sharey=True)
    df0o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]], style=ff_st_lines[ff_st[0]],
              label=ff_st[0], legend=False, ax=ax0)
    df1o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]], style=ff_st_lines[ff_st[1]],
              label=ff_st[1], legend=False, ax=ax0)
    df2o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]],  style=ff_st_lines[ff_st[2]],
              label=ff_st[2], legend=False, ax=ax0)
    df3o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]],  style=ff_st_lines[ff_st[3]],
              label=ff_st[3], legend=False, ax=ax0)
    ax0.set_xticks(df0o.tw)
    
    df0p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]], style=ff_st_lines[ff_st[0]],
              label=ff_st[0], legend=False, ax=ax1)
    df1p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]], style=ff_st_lines[ff_st[1]],
              label=ff_st[1], legend=False, ax=ax1)
    df2p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]], style=ff_st_lines[ff_st[2]],
              label=ff_st[2], legend=False, ax=ax1)
    df3p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
             c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]], style=ff_st_lines[ff_st[3]],
              label=ff_st[3], legend=False, ax=ax1)
    ax1.set_xticks(df0p.tw)
    
    for ax in [ax0, ax1]:
        ax.yaxis.set_minor_locator(plt.MultipleLocator(yax_minors[plots[j]]))
        ax.set_xticklabels([s for s in rs_tw], rotation=60, fontsize=12)
        ax.set_ylabel(ylab[plots[j]], fontsize=12)
        ax.set_xlabel('Intrinsic Twist ($^\circ$)')
        ax.invert_xaxis()
        ax.grid(axis='y',which='both')
    
    plt.legend(labels=[ff_st_dict[x] for x in ff_st], loc='center left', bbox_to_anchor=(1.05, 0.50))
    plt.tight_layout()
    plt.show()
    plt.clf()

del df0o, df1o, df2o, df3o, df0p, df1p, df2p, df3p, col150_df

In [ ]:
# Plot eopt, lk, wr, and rg for the 4 deltaTw Models

col150_df = pd.read_csv("data_col150_seq-primary_standardset", index_col=0)

for i in range(0, len(inseqs)):
    df0o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[0])]
    df1o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[1])]
    df2o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[2])]
    df3o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[3])]

    df0p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[0])]
    df1p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[1])]
    df2p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[2])]
    df3p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.seq==inseqs[i])&(col150_df.forcefield==ff_st[3])]

    for j in range(0, len(plots)):
        fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14,5), sharey=True)
        df0o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]], style=ff_st_lines[ff_st[0]],
                  label=ff_st[0], legend=False, ax=ax0)
        df1o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]], style=ff_st_lines[ff_st[1]],
                  label=ff_st[1], legend=False, ax=ax0)
        df2o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]], style=ff_st_lines[ff_st[2]],
                  label=ff_st[2], legend=False, ax=ax0)
        df3o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]], style=ff_st_lines[ff_st[3]],
                  label=ff_st[3], legend=False, ax=ax0)
        ax0.set_xticks(df0o.tw)

        df0p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]],  style=ff_st_lines[ff_st[0]],
                  label=ff_st[0], legend=False, ax=ax1)
        df1p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]],  style=ff_st_lines[ff_st[1]],
                  label=ff_st[1], legend=False, ax=ax1)
        df2p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]],  style=ff_st_lines[ff_st[2]],
                  label=ff_st[2], legend=False, ax=ax1)
        df3p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]],  style=ff_st_lines[ff_st[3]],
                  label=ff_st[3], legend=False, ax=ax1)
        ax1.set_xticks(df0p.tw)

        for ax in [ax0, ax1]:
            ax.yaxis.set_minor_locator(plt.MultipleLocator(yax_minors[plots[j]]))
            ax.set_xticklabels([s for s in rs_bpturn], rotation=60, fontsize=12)
            ax.set_ylabel(ylab[plots[j]], fontsize=12)
            ax.set_xlabel('Base-Pairs / Helical Turn')
            ax.grid(axis='y',which='both')

        plt.tight_layout()
        #plt.show()
        plt.savefig("col150_opt-deltaTw_"+plots[j]+"_"+inseqs[i]+'_v1', dpi=300, transparent=True)
        plt.clf()

    for j in range(0, len(plots)):
        fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14,5), sharey=True)
        df0o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]], style=ff_st_lines[ff_st[0]],
                  label=ff_st[0], legend=False, ax=ax0)
        df1o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]], style=ff_st_lines[ff_st[1]],
                  label=ff_st[1], legend=False, ax=ax0)
        df2o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]],  style=ff_st_lines[ff_st[2]],
                  label=ff_st[2], legend=False, ax=ax0)
        df3o.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]],  style=ff_st_lines[ff_st[3]],
                  label=ff_st[3], legend=False, ax=ax0)
        ax0.set_xticks(df0o.tw)

        df0p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[0]], marker=ff_st_marks[ff_st[0]], style=ff_st_lines[ff_st[0]],
                  label=ff_st[0], legend=False, ax=ax1)
        df1p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[1]], marker=ff_st_marks[ff_st[1]], style=ff_st_lines[ff_st[1]],
                  label=ff_st[1], legend=False, ax=ax1)
        df2p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[2]], marker=ff_st_marks[ff_st[2]], style=ff_st_lines[ff_st[2]],
                  label=ff_st[2], legend=False, ax=ax1)
        df3p.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                 c=ff_st_colors[ff_st[3]], marker=ff_st_marks[ff_st[3]], style=ff_st_lines[ff_st[3]],
                  label=ff_st[3], legend=False, ax=ax1)
        ax1.set_xticks(df0p.tw)

        for ax in [ax0, ax1]:
            ax.yaxis.set_minor_locator(plt.MultipleLocator(yax_minors[plots[j]]))
            ax.set_xticklabels([s for s in rs_tw], rotation=60, fontsize=12)
            ax.set_ylabel(ylab[plots[j]], fontsize=12)
            ax.set_xlabel('Intrinsic Twist ($^\circ$)')
            ax.invert_xaxis()
            ax.grid(axis='y',which='both')

        plt.tight_layout()
        #plt.show()
        plt.savefig("col150_opt-deltaTw_"+plots[j]+"_"+inseqs[i]+'_v2', dpi=300, transparent=True)
        plt.clf()

    del df0o, df1o, df2o, df3o, df0p, df1p, df2p, df3p
del col150_df

In [ ]:
# Plot eopt, lk, wr, and rg for the 4 deltaTw Models

col150_df = pd.read_csv("data_col150_seq-primary_standardset", index_col=0)

group0 = inseqs
group1 = ['col01','col02','col03']
group2 = ['col05','col06','col07','col08','col09']
group3 = ['col01','col03','col04','col09','col05']
groups = [group0, group1, group2, group3]

for g in range(0, len(groups)):
    for i in range(0, len(ff_st)):
        df0o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.forcefield==ff_st[i])]
        df0p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.forcefield==ff_st[i])]

        for j in range(0, len(plots)):
            fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14,5), sharey=True)
            for k in range(0, len(groups[g])):
                df0 = df0o.loc[(df0o.seq==groups[g][k])]
                df1 = df0p.loc[(df0p.seq==groups[g][k])]
                df0.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                         color=colors[groups[g][k]], marker=marks[groups[g][k]], style=':', ax=ax0, legend=False)
                df1.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                         color=colors[groups[g][k]], marker=marks[groups[g][k]], style='--', ax=ax1, legend=False)
                ax0.set_xticks(df0.tw)
                ax1.set_xticks(df1.tw)
                del df0, df1

            for ax in [ax0, ax1]:
                ax.yaxis.set_minor_locator(plt.MultipleLocator(yax_minors[plots[j]]))
                ax.set_xticklabels([s for s in rs_bpturn], rotation=60, fontsize=12)
                ax.set_ylabel(ylab[plots[j]], fontsize=12)
                ax.set_xlabel('Base-Pairs / Helical Turn')
                ax.grid(axis='y',which='both')

            plt.legend(labels=[labels[x] for x in groups[g]], loc='center left', bbox_to_anchor=(1.05, 0.50))
            plt.tight_layout()
            #plt.show()
            plt.savefig("col150_opt-deltaTw_"+plots[j]+"+"+str(ff_st[i])+"_group"+str(g)+"_v1", dpi=300, transparent=True)
            plt.clf()
        del df0o,df0p
    
    for i in range(0, len(ff_st)):
        df0o = col150_df.loc[(col150_df.incon==incons[0])&(col150_df.forcefield==ff_st[i])]
        df0p = col150_df.loc[(col150_df.incon==incons[1])&(col150_df.forcefield==ff_st[i])]

        for j in range(0, len(plots)):
            fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14,5), sharey=True)
            for k in range(0, len(groups[g])):
                df0 = df0o.loc[(df0o.seq==groups[g][k])]
                df1 = df0p.loc[(df0p.seq==groups[g][k])]
                df0.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                         color=colors[groups[g][k]], marker=marks[groups[g][k]], style=':', ax=ax0, legend=False)
                df1.plot(x='tw', y=plots[j], ylim=(ylimits[plots[j]][0],ylimits[plots[j]][1]),
                         color=colors[groups[g][k]], marker=marks[groups[g][k]], style='--', ax=ax1, legend=False)
                ax0.set_xticks(df0.tw)
                ax1.set_xticks(df1.tw)
                del df0, df1

            for ax in [ax0, ax1]:
                ax.yaxis.set_minor_locator(plt.MultipleLocator(yax_minors[plots[j]]))
                ax.set_xticklabels([s for s in rs_tw], rotation=60, fontsize=12)
                ax.set_ylabel(ylab[plots[j]], fontsize=12)
                ax.set_xlabel('Intrinsic Twist ($^\circ$)')
                ax.invert_xaxis()
                ax.grid(axis='y',which='both')

            plt.legend(labels=[labels[x] for x in groups[g]], loc='center left', bbox_to_anchor=(1.05, 0.50))
            plt.tight_layout()
            #plt.show()
            plt.savefig("col150_opt-deltaTw_"+plots[j]+"+"+str(ff_st[i])+"_group"+str(g)+"_v2", dpi=300, transparent=True)
            plt.clf()
        del df0o,df0p
    
del col150_df